In [2]:
'''
import packages we need
'''
import pandas as pd
import csv

In [3]:
import numpy as np

In [4]:
df = pd.read_csv("weight_all.csv")#, encoding='latin-1') 

/opt/miniconda3/envs/d2l/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [5]:
'''
show us the first five lines of assembled POIs
'''
df.head()

,步行最短距离（m）,步行最短时间（s）,公共服务设施名称,ID,距离分类,权重,纠正后权重
0,748,598,奇迹健身馆(西海岸店),1,0.55,0.08,0.044
1,748,598,心逸瑜伽(沙井总店),1,0.55,0.08,0.044
2,889,711,奇迹健身俱乐部(百利西城店),1,0.25,0.08,0.02
3,1102,882,加州阳光国际健身俱乐部(沙井店),1,0.08,0.08,0.0064
4,1237,990,拉伽瑜伽(沙井岁宝店),1,0.08,0.08,0.0064


In [6]:
dff = pd.read_csv("street_2.0.csv",header = None)

In [12]:
'''
assemble all pois of one address and calculate its grades,save it in dictionary 'data'
'''
data = {}
for i in range(1,230):#range(1,230)
    df_1 = df[(df['ID'] == i)]
    df_1c = df_1['纠正后权重']
    sum1 = 0
    for j in list(df_1c):
        try:
            i_f = float(j)
        except ValueError:
            i_f = float(j[:-2])
        sum1 += float(i_f)
    #print(sum1)
    data[i] = sum1

In [13]:
data

{1: 168.4572999999995,
 2: 362.90549999999865,
 3: 325.1469999999971,
 4: 373.76299999999725,
 5: 515.1739999999963,
 6: 225.80939999999933,
 7: 318.76449999999784,
 8: 756.3382000000123,
 9: 890.7273000000245,
 10: 1074.9911999999988,
 11: 304.27679999999833,
 12: 518.2403999999984,
 13: 821.4797000000112,
 14: 532.7147999999919,
 15: 559.6284999999987,
 16: 567.885399999999,
 17: 459.2045000000035,
 18: 408.2329999999983,
 19: 836.4674000000252,
 20: 346.55709999999425,
 21: 1033.401000000003,
 22: 1251.5511000000097,
 23: 431.3853999999946,
 24: 551.7119999999977,
 25: 1220.970800000012,
 26: 655.4640000000148,
 27: 390.2053999999946,
 28: 292.5829999999989,
 29: 417.50099999999907,
 30: 282.2243999999999,
 31: 505.45599999999723,
 32: 343.3449999999991,
 33: 415.11319999999495,
 34: 96.73449999999987,
 35: 259.40990000000113,
 36: 876.9678000000144,
 37: 824.3359000000086,
 38: 939.0144000000247,
 39: 896.2129000000097,
 40: 389.778999999999,
 41: 295.97199999999884,
 42: 190.49599

In [14]:
data[1]

168.4572999999995

In [15]:
max(data.values())

1354.9687000000183

In [16]:
min(data.values())

11.568800000000019

In [17]:
'''
show data in DataFrame format
'''
data_df = pd.DataFrame(data,index=[0]).T
data_df

,0
1,168.4573
2,362.9055
3,325.1470
4,373.7630
5,515.1740
...,...
225,80.3728
226,134.7521
227,34.6284
228,22.8825


In [18]:
data_df[0]

1      168.4573
2      362.9055
3      325.1470
4      373.7630
5      515.1740
         ...   
225     80.3728
226    134.7521
227     34.6284
228     22.8825
229     24.3330
Name: 0, Length: 229, dtype: float64

In [19]:
'''
here are some basic information of scores after distance-based decay function 
'''
data_df.describe()

,0
count,229.000000
mean,422.909595
std,295.567350
min,11.568800
25%,181.202900
50%,365.256000
75%,571.258800
max,1354.968700


In [20]:
'''
add a new column
'''
dff['3']=''

In [21]:
dff

,0,1,2,3
0,"113.78124,22.734616",沙井街道,"[65.96661453668062, 145.12748538961029]",
1,"113.791347,22.713674",沙井街道,"[65.96661453668062, 145.12748538961029]",
2,"113.822293,22.75981",沙井街道,"[65.96661453668062, 145.12748538961029]",
3,"113.812641,22.751043",沙井街道,"[65.96661453668062, 145.12748538961029]",
4,"113.802946,22.752249",沙井街道,"[65.96661453668062, 145.12748538961029]",
...,...,...,...,...
224,"114.485708,22.574515",大鹏街道,"[6.024117677770745, 254.6446409638549]",
225,"114.493143,22.532703",南澳街道,"[2.677708246753552, 327.19840322580626]",
226,"114.528983,22.553854",南澳街道,"[2.677708246753552, 327.19840322580626]",
227,"114.540863,22.55665",南澳街道,"[2.677708246753552, 327.19840322580626]",


In [22]:
'''
block-characteristic-based decay function
'''
def fix(x,y):
    if x < 50:
        wei = 0.95
    elif x < 80 and x >= 50:
        wei = 0.96
    elif x < 120 and x >= 80:
        wei = 0.97
    elif x < 150 and x >= 120:
        wei = 0.98
    elif x < 200 and x >= 150:
        wei = 0.99
    else:
        wei = 1

    if y < 100:
        ght = 1
    elif x < 120 and x >= 100:
        ght = 0.99
    elif x < 150 and x >= 120:
        ght = 0.98
    elif x < 180 and x >= 150:
        ght = 0.97
    elif x < 200 and x >= 180:
        ght = 0.96
    else:
        ght = 0.95
    return(wei*ght)

In [23]:
'''
calculate scores after considering block characteristics
'''
for i in data:
    dff.iloc[[i-1],[3]]=data[i]
    line = dff.iat[i-1, 2]
    line = line.replace('[', '')
    line = line.replace(']', '')
    line = line.split(',')
    #print(line)
    x = float(line[0])
    y = float(line[1])
    dff.iloc[[i-1],[3]]=data[i]*fix(x,y)

In [24]:
'''
show us the first five lines of assembled POIs
'''
dff.head()

,0,1,2,3
0,"113.78124,22.734616",沙井街道,"[65.96661453668062, 145.12748538961029]",153.633058
1,"113.791347,22.713674",沙井街道,"[65.96661453668062, 145.12748538961029]",330.969816
2,"113.822293,22.75981",沙井街道,"[65.96661453668062, 145.12748538961029]",296.534064
3,"113.812641,22.751043",沙井街道,"[65.96661453668062, 145.12748538961029]",340.871856
4,"113.802946,22.752249",沙井街道,"[65.96661453668062, 145.12748538961029]",469.838688


In [25]:
'''
save file
'''
dff.to_csv("score.csv",index=False,sep=',')

In [36]:
max(dff['3'])

1273.626392999984

In [27]:
'''
extract final scores 
'''
dff_1=pd.DataFrame(dff['3'],dtype=np.float)

In [28]:
'''
show us basic information of the final scores
'''
dff_1.describe()

,3
count,229.000000
mean,387.549526
std,274.228856
min,10.440842
25%,163.535617
50%,330.969816
75%,520.988026
max,1273.626393


In [29]:
'''
only keep three digital numbers
'''
dff_1.round(3).describe()

,3
count,229.000000
mean,387.549502
std,274.228853
min,10.441000
25%,163.536000
50%,330.970000
75%,520.988000
max,1273.626000


In [30]:
#中位数
np.median(dff_1.round(3))

330.97

In [31]:
from scipy import stats

In [32]:
'''
use stats package to calculate mode
'''
stats.mode(dff_1.round(0))[0][0]

array([492.])

In [37]:
'''
a new score.csv, after using Excel to standardize scores into [0,100]
'''
score = pd.read_csv("score.csv")

In [38]:
score.head()

,0,1,2,3,4
0,"113.78124,22.734616",沙井街道,"[65.96661453668062, 145.12748538961029]",153.633058,11.335802
1,"113.791347,22.713674",沙井街道,"[65.96661453668062, 145.12748538961029]",330.969816,25.374655
2,"113.822293,22.75981",沙井街道,"[65.96661453668062, 145.12748538961029]",296.534064,22.648551
3,"113.812641,22.751043",沙井街道,"[65.96661453668062, 145.12748538961029]",340.871856,26.158549
4,"113.802946,22.752249",沙井街道,"[65.96661453668062, 145.12748538961029]",469.838688,36.368200


In [39]:
'''
show us basic statistic information about standardized final scores
'''
score['4'].round(2).describe()

count    229.000000
mean      29.853930
std       21.709345
min        0.000000
25%       12.120000
50%       25.370000
75%       40.420000
max      100.000000
Name: 4, dtype: float64

In [40]:
np.median(score['4'].round(2))

25.37

In [41]:
stats.mode(score['4'].round(0))[0][0]

35.0

In [42]:
'''
calculate the numbers of communities that fall into different score scope
'''
#：[(10, 20] < (20, 30] < (30, 50] < (50, 70] < (70, 80] < (80, 90]]
scores_bins = [-0.01,24.99,49.99,69.99,89.99,100.01]
a,b = pd.cut(x=score["4"],bins=scores_bins,right=True,retbins=True)

In [43]:
a

0      (-0.01, 24.99]
1      (24.99, 49.99]
2      (-0.01, 24.99]
3      (24.99, 49.99]
4      (24.99, 49.99]
            ...      
224    (-0.01, 24.99]
225    (-0.01, 24.99]
226    (-0.01, 24.99]
227    (-0.01, 24.99]
228    (-0.01, 24.99]
Name: 4, Length: 229, dtype: category
Categories (5, interval[float64]): [(-0.01, 24.99] < (24.99, 49.99] < (49.99, 69.99] < (69.99, 89.99] < (89.99, 100.01]]

In [44]:
a.value_counts()

(-0.01, 24.99]     113
(24.99, 49.99]      71
(49.99, 69.99]      33
(69.99, 89.99]      10
(89.99, 100.01]      2
Name: 4, dtype: int64